In [1]:
import pandas as pd
import numpy as np
import h5py
import time 

In [2]:
def get_2D_data_from_h5_filtered(h5_path, part_name, Slice_name, mode):
    #Step 1: getting the data from the h5
    start_time = time.time()
    with h5py.File(h5_path,'r') as h5:
        X_Axis = h5[part_name][Slice_name]['X-Axis']
        Y_Axis = h5[part_name][Slice_name]['Y-Axis']
        Area = h5[part_name][Slice_name]['Area']
        Intensity = h5[part_name][Slice_name]['Intensity']

        X_Axis_size = X_Axis.size
        Y_Axis_size = Y_Axis.size
        Area_size = Area.size
        Intensity_size = Intensity.size

        #if dimensions aren't equal the following code block is entered
        if not X_Axis_size == Y_Axis_size == Area_size == Intensity_size:

            #determine the lowest value among the different sizes
            size_arr = np.array([X_Axis_size, Y_Axis_size, Area_size, Intensity_size])
            min_size = size_arr.min()

            if X_Axis_size != min_size:
                diff_size_x = X_Axis_size - min_size #calculating the difference between the actual value and the minimum and substracting it from the array
                X_Axis_new = np.delete(X_Axis, -diff_size_x)
                X_Axis = X_Axis_new
                X_Axis_size = X_Axis.size

            if Y_Axis_size != min_size:
                diff_size_y = Y_Axis_size - min_size
                Y_Axis_new = np.delete(Y_Axis, -diff_size_y)
                Y_Axis = Y_Axis_new
                Y_Axis_size = Y_Axis.size

            if Area_size != min_size:
                diff_size_area = Area_size - min_size
                Area_new = np.delete(Area, -diff_size_area)
                Area = Area_new
                Area_size = Area.size

            if Intensity_size != min_size:
                diff_size_intensity = Intensity_size - min_size
                Intensity_new = np.delete(Intensity, -diff_size_intensity)
                Intensity = Intensity_new
                Intensity_size = Intensity.size


        #by reducing all the dimensions to the minimum equal dimensions are guaranteed
        #there is a risk of deleting more than just one datapoint without noticing -> maybe add an alert after more than 5(?) while iterations
        help_arr = np.column_stack((X_Axis, Y_Axis, Area, Intensity))
        df_raw = pd.DataFrame(help_arr, columns=['x','y','area','intensity'])

    #Step 2: change floats to ints and remove duplicates
    df_int = df_raw.astype(int).drop_duplicates()

    #remove all rows with 0 for area and intensity
    df_int = df_int.loc[(df_int['area'] != 0) & (df_int['intensity'] != 0)]


    #Step 3: Get a df with all the rows where a certain x,y combination occurs multiple times
    df_multi_xy = df_int[df_int.duplicated(['x','y'], keep = False)]

    #Step 4: get a new df out of df_multi_xy with x,y and mean/max of area and intensity for all x,y occurences
    df_compact = pd.DataFrame(columns=['x','y','area','intensity']) #initialize df_compact

    print("vor iterieren %s seconds ---" % (time.time() - start_time))
    for ind in range (df_multi_xy.shape[0]):
        if mode == 'mean':
            area_mean = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['area'].mean().astype(int)
            intensity_mean = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['intensity'].mean().astype(int)
            df_compact = df_compact.append({'x': df_multi_xy.iloc[ind]['x'], 'y':df_multi_xy.iloc[ind]['y'], 'area':area_mean , 'intensity':intensity_mean}, ignore_index=True)
        if mode == 'max':
            area_max = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['area'].max().astype(int)
            intensity_max = df_multi_xy.loc[(df_multi_xy['x']== df_multi_xy.iloc[ind]['x']) & (df_multi_xy['y'] == df_multi_xy.iloc[ind]['y'])]['intensity'].max().astype(int)
            df_compact = df_compact.append({'x': df_multi_xy.iloc[ind]['x'], 'y':df_multi_xy.iloc[ind]['y'], 'area':area_max , 'intensity':intensity_max}, ignore_index=True)
    df_compact = df_compact.drop_duplicates()

    #Step 5: remove df_multi_xy from df_int and append df_compact
    df_multi_xy_removed = df_int.drop(df_int[df_int.duplicated(['x','y'], keep = False)].index)

    df_final = df_multi_xy_removed.append(df_compact)
    print("df creation took %s seconds ---" % (time.time() - start_time))
    return (df_final)

In [3]:
df_undocked = get_2D_data_from_h5_filtered('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5', '0_00003_Canti3_cls', 'Slice00003', 'mean')

vor iterieren 0.06304788589477539 seconds ---
df creation took 77.99655294418335 seconds ---


In [5]:
def get_true_min_maxX (h5_path, part_name, max_slice_number):

    minX = []
    maxX = []
    for num_slice in range(max_slice_number):
        with h5py.File(h5_path,'r') as h5:
            X_Axis = h5[part_name]['Slice'+str("{:05d}".format(num_slice+1))]['X-Axis']
            x_axis_array = np.array(X_Axis)
            minX.append(x_axis_array.min())
            maxX.append(x_axis_array.max())
    minX_array = np.asarray(minX)
    maxX_array = np.asarray(maxX)
    return minX_array.min(), maxX_array.max()

In [6]:
def get_true_min_maxY (h5_path, part_name, max_slice_number):

    minY = []
    maxY = []
    for num_slice in range(max_slice_number):
        with h5py.File(h5_path,'r') as h5:
            Y_Axis = h5[part_name]['Slice'+str("{:05d}".format(num_slice+1))]['Y-Axis']
            y_axis_array = np.array(Y_Axis)
            minY.append(y_axis_array.min())
            maxY.append(y_axis_array.max())
    minY_array = np.asarray(minY)
    maxY_array = np.asarray(maxY)
    return minY_array.min(), maxY_array.max()

In [7]:
minX = get_true_min_maxX('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5', '0_00003_Canti3_cls', 142)[0]
maxX = get_true_min_maxX('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5', '0_00003_Canti3_cls', 142)[1]
minY = get_true_min_maxY('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5', '0_00003_Canti3_cls', 142)[0]
maxY = get_true_min_maxY('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5', '0_00003_Canti3_cls', 142)[1]

In [4]:
def dock_df_to_zero(df, minX, minY):
    if minX >= 0 and minY >=0:
        df['x'] = df['x'] - minX
        df['y'] = df['y'] - minY
    elif minX < 0 and minY <0:
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] + abs(minY)
    elif minX >= 0 and minY <0:
        df['x'] = df['x'] - minX
        df['y'] = df['y'] + abs(minY)
    elif minX < 0 and min >= 0:
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] - minY
    return df

In [8]:
df = dock_df_to_zero(df_undocked, minX, minY)

In [9]:
df

,x,y,area,intensity
971,3519,3578,203,545
973,3535,3578,248,520
974,3541,3578,221,521
975,3551,3578,194,1017
976,3557,3578,265,932
...,...,...,...,...
17075,3528,8817,298,530
17082,3649,8977,347,577
17088,3528,8897,240,504
17095,3569,8977,183,474


In [21]:
def fill_2D_voxel_area_v2 (voxel_size, num_voxels_x, num_voxels_y, df):
    start_time = time.time()
    array_area = np.zeros([voxel_size,voxel_size]) #creating an empty array of dimensions voxel_size*voxel_size
    for i in range(voxel_size*num_voxels_x, voxel_size*(num_voxels_x+1)): #iterating over x
        for j in range(voxel_size*num_voxels_y,voxel_size*(num_voxels_y+1)): #iterating over y
            print ('x: ' + str(i))
            print('y: ' + str(j))
            if ((df['x'] == i) & (df['y'] == j)).any(): #here subset of the original dataframe is created an filtrered --> shape[0] returns the number of rows of this df subset
                #finding the area value for a certain point in the part-data-dataframe and allocating it to a position in the array
                area_i = df.loc[(df['x'] == i) & (df['y'] == j)]
                array_area[i-num_voxels_x*voxel_size][j-num_voxels_y*voxel_size] = area_i['area']

            else:
                array_area[i-num_voxels_x*voxel_size][j-num_voxels_y*voxel_size] = 0
        
    print("voxel took %s seconds ---" % (time.time() - start_time))
    print(array_area)

    return array_area
      

In [22]:
fill_2D_voxel_area_v2(100, 35, 35, df)

x: 3500
y: 3500
x: 3500
y: 3501
x: 3500
y: 3502
x: 3500
y: 3503
x: 3500
y: 3504
x: 3500
y: 3505
x: 3500
y: 3506
x: 3500
y: 3507
x: 3500
y: 3508
x: 3500
y: 3509
x: 3500
y: 3510
x: 3500
y: 3511
x: 3500
y: 3512
x: 3500
y: 3513
x: 3500
y: 3514
x: 3500
y: 3515
x: 3500
y: 3516
x: 3500
y: 3517
x: 3500
y: 3518
x: 3500
y: 3519
x: 3500
y: 3520
x: 3500
y: 3521
x: 3500
y: 3522
x: 3500
y: 3523
x: 3500
y: 3524
x: 3500
y: 3525
x: 3500
y: 3526
x: 3500
y: 3527
x: 3500
y: 3528
x: 3500
y: 3529
x: 3500
y: 3530
x: 3500
y: 3531
x: 3500
y: 3532
x: 3500
y: 3533
x: 3500
y: 3534
x: 3500
y: 3535
x: 3500
y: 3536
x: 3500
y: 3537
x: 3500
y: 3538
x: 3500
y: 3539
x: 3500
y: 3540
x: 3500
y: 3541
x: 3500
y: 3542
x: 3500
y: 3543
x: 3500
y: 3544
x: 3500
y: 3545
x: 3500
y: 3546
x: 3500
y: 3547
x: 3500
y: 3548
x: 3500
y: 3549
x: 3500
y: 3550
x: 3500
y: 3551
x: 3500
y: 3552
x: 3500
y: 3553
x: 3500
y: 3554
x: 3500
y: 3555
x: 3500
y: 3556
x: 3500
y: 3557
x: 3500
y: 3558
x: 3500
y: 3559
x: 3500
y: 3560
x: 3500
y: 3561
x: 3500


x: 3505
y: 3525
x: 3505
y: 3526
x: 3505
y: 3527
x: 3505
y: 3528
x: 3505
y: 3529
x: 3505
y: 3530
x: 3505
y: 3531
x: 3505
y: 3532
x: 3505
y: 3533
x: 3505
y: 3534
x: 3505
y: 3535
x: 3505
y: 3536
x: 3505
y: 3537
x: 3505
y: 3538
x: 3505
y: 3539
x: 3505
y: 3540
x: 3505
y: 3541
x: 3505
y: 3542
x: 3505
y: 3543
x: 3505
y: 3544
x: 3505
y: 3545
x: 3505
y: 3546
x: 3505
y: 3547
x: 3505
y: 3548
x: 3505
y: 3549
x: 3505
y: 3550
x: 3505
y: 3551
x: 3505
y: 3552
x: 3505
y: 3553
x: 3505
y: 3554
x: 3505
y: 3555
x: 3505
y: 3556
x: 3505
y: 3557
x: 3505
y: 3558
x: 3505
y: 3559
x: 3505
y: 3560
x: 3505
y: 3561
x: 3505
y: 3562
x: 3505
y: 3563
x: 3505
y: 3564
x: 3505
y: 3565
x: 3505
y: 3566
x: 3505
y: 3567
x: 3505
y: 3568
x: 3505
y: 3569
x: 3505
y: 3570
x: 3505
y: 3571
x: 3505
y: 3572
x: 3505
y: 3573
x: 3505
y: 3574
x: 3505
y: 3575
x: 3505
y: 3576
x: 3505
y: 3577
x: 3505
y: 3578
x: 3505
y: 3579
x: 3505
y: 3580
x: 3505
y: 3581
x: 3505
y: 3582
x: 3505
y: 3583
x: 3505
y: 3584
x: 3505
y: 3585
x: 3505
y: 3586
x: 3505


x: 3510
y: 3538
x: 3510
y: 3539
x: 3510
y: 3540
x: 3510
y: 3541
x: 3510
y: 3542
x: 3510
y: 3543
x: 3510
y: 3544
x: 3510
y: 3545
x: 3510
y: 3546
x: 3510
y: 3547
x: 3510
y: 3548
x: 3510
y: 3549
x: 3510
y: 3550
x: 3510
y: 3551
x: 3510
y: 3552
x: 3510
y: 3553
x: 3510
y: 3554
x: 3510
y: 3555
x: 3510
y: 3556
x: 3510
y: 3557
x: 3510
y: 3558
x: 3510
y: 3559
x: 3510
y: 3560
x: 3510
y: 3561
x: 3510
y: 3562
x: 3510
y: 3563
x: 3510
y: 3564
x: 3510
y: 3565
x: 3510
y: 3566
x: 3510
y: 3567
x: 3510
y: 3568
x: 3510
y: 3569
x: 3510
y: 3570
x: 3510
y: 3571
x: 3510
y: 3572
x: 3510
y: 3573
x: 3510
y: 3574
x: 3510
y: 3575
x: 3510
y: 3576
x: 3510
y: 3577
x: 3510
y: 3578
x: 3510
y: 3579
x: 3510
y: 3580
x: 3510
y: 3581
x: 3510
y: 3582
x: 3510
y: 3583
x: 3510
y: 3584
x: 3510
y: 3585
x: 3510
y: 3586
x: 3510
y: 3587
x: 3510
y: 3588
x: 3510
y: 3589
x: 3510
y: 3590
x: 3510
y: 3591
x: 3510
y: 3592
x: 3510
y: 3593
x: 3510
y: 3594
x: 3510
y: 3595
x: 3510
y: 3596
x: 3510
y: 3597
x: 3510
y: 3598
x: 3510
y: 3599
x: 3511


x: 3515
y: 3565
x: 3515
y: 3566
x: 3515
y: 3567
x: 3515
y: 3568
x: 3515
y: 3569
x: 3515
y: 3570
x: 3515
y: 3571
x: 3515
y: 3572
x: 3515
y: 3573
x: 3515
y: 3574
x: 3515
y: 3575
x: 3515
y: 3576
x: 3515
y: 3577
x: 3515
y: 3578
x: 3515
y: 3579
x: 3515
y: 3580
x: 3515
y: 3581
x: 3515
y: 3582
x: 3515
y: 3583
x: 3515
y: 3584
x: 3515
y: 3585
x: 3515
y: 3586
x: 3515
y: 3587
x: 3515
y: 3588
x: 3515
y: 3589
x: 3515
y: 3590
x: 3515
y: 3591
x: 3515
y: 3592
x: 3515
y: 3593
x: 3515
y: 3594
x: 3515
y: 3595
x: 3515
y: 3596
x: 3515
y: 3597
x: 3515
y: 3598
x: 3515
y: 3599
x: 3516
y: 3500
x: 3516
y: 3501
x: 3516
y: 3502
x: 3516
y: 3503
x: 3516
y: 3504
x: 3516
y: 3505
x: 3516
y: 3506
x: 3516
y: 3507
x: 3516
y: 3508
x: 3516
y: 3509
x: 3516
y: 3510
x: 3516
y: 3511
x: 3516
y: 3512
x: 3516
y: 3513
x: 3516
y: 3514
x: 3516
y: 3515
x: 3516
y: 3516
x: 3516
y: 3517
x: 3516
y: 3518
x: 3516
y: 3519
x: 3516
y: 3520
x: 3516
y: 3521
x: 3516
y: 3522
x: 3516
y: 3523
x: 3516
y: 3524
x: 3516
y: 3525
x: 3516
y: 3526
x: 3516


x: 3520
y: 3591
x: 3520
y: 3592
x: 3520
y: 3593
x: 3520
y: 3594
x: 3520
y: 3595
x: 3520
y: 3596
x: 3520
y: 3597
x: 3520
y: 3598
x: 3520
y: 3599
x: 3521
y: 3500
x: 3521
y: 3501
x: 3521
y: 3502
x: 3521
y: 3503
x: 3521
y: 3504
x: 3521
y: 3505
x: 3521
y: 3506
x: 3521
y: 3507
x: 3521
y: 3508
x: 3521
y: 3509
x: 3521
y: 3510
x: 3521
y: 3511
x: 3521
y: 3512
x: 3521
y: 3513
x: 3521
y: 3514
x: 3521
y: 3515
x: 3521
y: 3516
x: 3521
y: 3517
x: 3521
y: 3518
x: 3521
y: 3519
x: 3521
y: 3520
x: 3521
y: 3521
x: 3521
y: 3522
x: 3521
y: 3523
x: 3521
y: 3524
x: 3521
y: 3525
x: 3521
y: 3526
x: 3521
y: 3527
x: 3521
y: 3528
x: 3521
y: 3529
x: 3521
y: 3530
x: 3521
y: 3531
x: 3521
y: 3532
x: 3521
y: 3533
x: 3521
y: 3534
x: 3521
y: 3535
x: 3521
y: 3536
x: 3521
y: 3537
x: 3521
y: 3538
x: 3521
y: 3539
x: 3521
y: 3540
x: 3521
y: 3541
x: 3521
y: 3542
x: 3521
y: 3543
x: 3521
y: 3544
x: 3521
y: 3545
x: 3521
y: 3546
x: 3521
y: 3547
x: 3521
y: 3548
x: 3521
y: 3549
x: 3521
y: 3550
x: 3521
y: 3551
x: 3521
y: 3552
x: 3521


...

x: 3583
y: 3590
x: 3583
y: 3591
x: 3583
y: 3592
x: 3583
y: 3593
x: 3583
y: 3594
x: 3583
y: 3595
x: 3583
y: 3596
x: 3583
y: 3597
x: 3583
y: 3598
x: 3583
y: 3599
x: 3584
y: 3500
x: 3584
y: 3501
x: 3584
y: 3502
x: 3584
y: 3503
x: 3584
y: 3504
x: 3584
y: 3505
x: 3584
y: 3506
x: 3584
y: 3507
x: 3584
y: 3508
x: 3584
y: 3509
x: 3584
y: 3510
x: 3584
y: 3511
x: 3584
y: 3512
x: 3584
y: 3513
x: 3584
y: 3514
x: 3584
y: 3515
x: 3584
y: 3516
x: 3584
y: 3517
x: 3584
y: 3518
x: 3584
y: 3519
x: 3584
y: 3520
x: 3584
y: 3521
x: 3584
y: 3522
x: 3584
y: 3523
x: 3584
y: 3524
x: 3584
y: 3525
x: 3584
y: 3526
x: 3584
y: 3527
x: 3584
y: 3528
x: 3584
y: 3529
x: 3584
y: 3530
x: 3584
y: 3531
x: 3584
y: 3532
x: 3584
y: 3533
x: 3584
y: 3534
x: 3584
y: 3535
x: 3584
y: 3536
x: 3584
y: 3537
x: 3584
y: 3538
x: 3584
y: 3539
x: 3584
y: 3540
x: 3584
y: 3541
x: 3584
y: 3542
x: 3584
y: 3543
x: 3584
y: 3544
x: 3584
y: 3545
x: 3584
y: 3546
x: 3584
y: 3547
x: 3584
y: 3548
x: 3584
y: 3549
x: 3584
y: 3550
x: 3584
y: 3551
x: 3584


x: 3589
y: 3503
x: 3589
y: 3504
x: 3589
y: 3505
x: 3589
y: 3506
x: 3589
y: 3507
x: 3589
y: 3508
x: 3589
y: 3509
x: 3589
y: 3510
x: 3589
y: 3511
x: 3589
y: 3512
x: 3589
y: 3513
x: 3589
y: 3514
x: 3589
y: 3515
x: 3589
y: 3516
x: 3589
y: 3517
x: 3589
y: 3518
x: 3589
y: 3519
x: 3589
y: 3520
x: 3589
y: 3521
x: 3589
y: 3522
x: 3589
y: 3523
x: 3589
y: 3524
x: 3589
y: 3525
x: 3589
y: 3526
x: 3589
y: 3527
x: 3589
y: 3528
x: 3589
y: 3529
x: 3589
y: 3530
x: 3589
y: 3531
x: 3589
y: 3532
x: 3589
y: 3533
x: 3589
y: 3534
x: 3589
y: 3535
x: 3589
y: 3536
x: 3589
y: 3537
x: 3589
y: 3538
x: 3589
y: 3539
x: 3589
y: 3540
x: 3589
y: 3541
x: 3589
y: 3542
x: 3589
y: 3543
x: 3589
y: 3544
x: 3589
y: 3545
x: 3589
y: 3546
x: 3589
y: 3547
x: 3589
y: 3548
x: 3589
y: 3549
x: 3589
y: 3550
x: 3589
y: 3551
x: 3589
y: 3552
x: 3589
y: 3553
x: 3589
y: 3554
x: 3589
y: 3555
x: 3589
y: 3556
x: 3589
y: 3557
x: 3589
y: 3558
x: 3589
y: 3559
x: 3589
y: 3560
x: 3589
y: 3561
x: 3589
y: 3562
x: 3589
y: 3563
x: 3589
y: 3564
x: 3589


x: 3594
y: 3533
x: 3594
y: 3534
x: 3594
y: 3535
x: 3594
y: 3536
x: 3594
y: 3537
x: 3594
y: 3538
x: 3594
y: 3539
x: 3594
y: 3540
x: 3594
y: 3541
x: 3594
y: 3542
x: 3594
y: 3543
x: 3594
y: 3544
x: 3594
y: 3545
x: 3594
y: 3546
x: 3594
y: 3547
x: 3594
y: 3548
x: 3594
y: 3549
x: 3594
y: 3550
x: 3594
y: 3551
x: 3594
y: 3552
x: 3594
y: 3553
x: 3594
y: 3554
x: 3594
y: 3555
x: 3594
y: 3556
x: 3594
y: 3557
x: 3594
y: 3558
x: 3594
y: 3559
x: 3594
y: 3560
x: 3594
y: 3561
x: 3594
y: 3562
x: 3594
y: 3563
x: 3594
y: 3564
x: 3594
y: 3565
x: 3594
y: 3566
x: 3594
y: 3567
x: 3594
y: 3568
x: 3594
y: 3569
x: 3594
y: 3570
x: 3594
y: 3571
x: 3594
y: 3572
x: 3594
y: 3573
x: 3594
y: 3574
x: 3594
y: 3575
x: 3594
y: 3576
x: 3594
y: 3577
x: 3594
y: 3578
x: 3594
y: 3579
x: 3594
y: 3580
x: 3594
y: 3581
x: 3594
y: 3582
x: 3594
y: 3583
x: 3594
y: 3584
x: 3594
y: 3585
x: 3594
y: 3586
x: 3594
y: 3587
x: 3594
y: 3588
x: 3594
y: 3589
x: 3594
y: 3590
x: 3594
y: 3591
x: 3594
y: 3592
x: 3594
y: 3593
x: 3594
y: 3594
x: 3594


x: 3599
y: 3563
x: 3599
y: 3564
x: 3599
y: 3565
x: 3599
y: 3566
x: 3599
y: 3567
x: 3599
y: 3568
x: 3599
y: 3569
x: 3599
y: 3570
x: 3599
y: 3571
x: 3599
y: 3572
x: 3599
y: 3573
x: 3599
y: 3574
x: 3599
y: 3575
x: 3599
y: 3576
x: 3599
y: 3577
x: 3599
y: 3578
x: 3599
y: 3579
x: 3599
y: 3580
x: 3599
y: 3581
x: 3599
y: 3582
x: 3599
y: 3583
x: 3599
y: 3584
x: 3599
y: 3585
x: 3599
y: 3586
x: 3599
y: 3587
x: 3599
y: 3588
x: 3599
y: 3589
x: 3599
y: 3590
x: 3599
y: 3591
x: 3599
y: 3592
x: 3599
y: 3593
x: 3599
y: 3594
x: 3599
y: 3595
x: 3599
y: 3596
x: 3599
y: 3597
x: 3599
y: 3598
x: 3599
y: 3599
voxel took 103.55628681182861 seconds ---
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
#list with combinations for the selected voxel 
x_values = np.repeat(np.arange(3500,3600,1),100)          #np.repeat(np.arange(num_voxels_x*voxel_size,(num_voxels_x+1)*voxel_size,1),(num_voxels_x+1)*voxel_size)
y_values = np.tile(np.arange(2500,2600,1),100)

In [45]:
x_values

array([3500, 3500, 3500, ..., 3599, 3599, 3599])

In [46]:
y_values

array([2500, 2501, 2502, ..., 2597, 2598, 2599])

In [52]:
combined = np.stack((x_values, y_values), axis=-1)

In [53]:
combined

array([[3500, 2500],
       [3500, 2501],
       [3500, 2502],
       ...,
       [3599, 2597],
       [3599, 2598],
       [3599, 2599]])

In [56]:
combined.tolist()

[[3500, 2500],
 [3500, 2501],
 [3500, 2502],
 [3500, 2503],
 [3500, 2504],
 [3500, 2505],
 [3500, 2506],
 [3500, 2507],
 [3500, 2508],
 [3500, 2509],
 [3500, 2510],
 [3500, 2511],
 [3500, 2512],
 [3500, 2513],
 [3500, 2514],
 [3500, 2515],
 [3500, 2516],
 [3500, 2517],
 [3500, 2518],
 [3500, 2519],
 [3500, 2520],
 [3500, 2521],
 [3500, 2522],
 [3500, 2523],
 [3500, 2524],
 [3500, 2525],
 [3500, 2526],
 [3500, 2527],
 [3500, 2528],
 [3500, 2529],
 [3500, 2530],
 [3500, 2531],
 [3500, 2532],
 [3500, 2533],
 [3500, 2534],
 [3500, 2535],
 [3500, 2536],
 [3500, 2537],
 [3500, 2538],
 [3500, 2539],
 [3500, 2540],
 [3500, 2541],
 [3500, 2542],
 [3500, 2543],
 [3500, 2544],
 [3500, 2545],
 [3500, 2546],
 [3500, 2547],
 [3500, 2548],
 [3500, 2549],
 [3500, 2550],
 [3500, 2551],
 [3500, 2552],
 [3500, 2553],
 [3500, 2554],
 [3500, 2555],
 [3500, 2556],
 [3500, 2557],
 [3500, 2558],
 [3500, 2559],
 [3500, 2560],
 [3500, 2561],
 [3500, 2562],
 [3500, 2563],
 [3500, 2564],
 [3500, 2565],
 [3500, 25

In [67]:
start_time = time.time()
array_area = np.zeros([100,100])
for value_pair in combined:
    #print(value_pair[0])
    #print(value_pair[1])
    if ((df['x'] == value_pair[0]) & (df['y'] == value_pair[1])).any(): #here subset of the original dataframe is created an filtrered --> shape[0] returns the number of rows of this df subset
        #finding the area value for a certain point in the part-data-dataframe and allocating it to a position in the array
        array_area[value_pair[0]-3500][value_pair[1]-2500] = df.loc[(df['x'] == value_pair[0]) & (df['y'] == value_pair[1])]['area']
    else:
        array_area[value_pair[0]-3500][value_pair[1]-2500] = 0

print(array_area)
print("voxel took %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [ ]:
#conclusion: working with lsit doenst really speed up the whole process